# Titanic

**Titanic** alebo **RMS Titanic** (*RMS* je skratka pre *Royal Mail Ship* – *Kráľovský poštový parník*) bol luxusný zaoceánsky parník triedy *Olympic* patriaci spoločnosti *White Star Line*. Jeho úlohou mal byť prevoz cestujúcich a pošty medzi Európou a Severnou Amerikou. Kapacita lode dovoľovala nalodenie *2 453* až *2 603* cestujúcich, batožiny a obmedzený počet kočiarov alebo automobilov. O prevádzku lode a o pohodlie cestujúcich sa staralo *885* až *899* členov posádky. 

![Titanic](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/RMS_Titanic_3.jpg/1024px-RMS_Titanic_3.jpg)

Titanic však stroskotal už počas svojej prvej plavby v noci zo *14.* na *15. apríla* roku *1912*. Zahynulo okolo *1 450* cestujúcich a členov posádky.

V tomto labe sa pozrieme na katastrofu cez čísla. Dataset je ukradnutý zo stránok [Kaggle](https://www.kaggle.com/competitions/titanic), kde slúži ako vstupné zadanie do sveta strojového učenia.

## 1. Základné informácie

V tomto kroku sa pozrieme na to, kto sa vlastne na *Titaniku* viezol v kritickú a vlastne jedinú plavbu.

### 1.1 Vytvorte dataset

Vytvorte dataset so zoznamom všetkých pasažierov. Tento dataset zostavte z dvoch datasetov s názvom `test.csv` a `train.csv`, ktoré sa nachádzajú uložené v priečinku `data/titanic/`. Index zostavte zo stĺpca `PassengerId`.

Výsledný dataset uložte do premennej `passengers`.

In [34]:
# solution
import pandas as pd

df_test = pd.read_csv('data/titanic/test.csv', index_col='PassengerId')
df_train = pd.read_csv('data/titanic/train.csv', index_col='PassengerId')

passengers = pd.concat([df_train, df_test])

### 1.2 Koľko bolo cestujúcich?

Zistite, koľko cestujúcich sa zúčastnilo plavby.

In [51]:
# solution
len(passengers)
passengers['Name'].count()
len(passengers.index)
passengers.index.max()
passengers.shape[0]

1309

### 1.3 Koľko bolo na palube žien a koľko mužov?

Zistite, koľko bolo na palube žien a mužov. Výsledok reprezentujte vo forme slovníka v tvare:

```json
{
    'males': 123,
    'females': 456
}
```

In [65]:
# solution
is_woman = (passengers['Sex'] == 'female')
is_man = (passengers['Sex'] == 'male')

{
    'female': passengers.loc[ is_woman, 'Name' ].count(),
    'male': passengers.loc[ is_man, 'Name' ].count()
}

passengers.groupby('Sex')['Sex'].count().to_dict()

{'female': 466, 'male': 843}

### 1.4 Koľko bolo na palube detí?

Za dieťa považujeme osobu, ktorá nedosiahla vek *15* rokov.

In [68]:
# solution
is_kid = (passengers['Age'] < 15)

passengers.loc[is_kid, ['Age']].count()

Age    109
dtype: int64

### 1.5 Najstarší pasažier

Ktorý z cestujúcich bol najstarším pasažierom? Okrem najstaršieho cestujúceho celkovo vypíšte aj najstaršiu cestujúcu a najstaršieho cestujúceho. Vždy o cestujúcom vypíšte meno, vek a pohlavie.

In [104]:
# solution
fltr = passengers['Age'] == passengers['Age'].max()
passengers.loc[ fltr, ['Name', 'Age'] ]

is_female = (passengers['Sex'] == 'female')
is_male = (passengers['Sex'] == 'male')

pd.concat([
    # oldest female
    passengers.loc[ is_female, ['Name', 'Age', 'Sex']] \
        .nlargest(1, 'Age', keep='all') \
        .dropna()
,
    # oldest male
    passengers.loc[ is_male, ['Name', 'Age', 'Sex']] \
        .nlargest(1, 'Age', keep='all') \
        .dropna()
]) 

,Name,Sex,Age
PassengerId,,,
988,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0


### 1.6 Najmladší pasažier

A naopak - ktorý z cestujúcich bol najmladší? Okrem najmladšieho cestujúceho celkovo vypíšte aj najmladšiu cestujúcu a najmladšieho cestujúceho. Vždy vypíšte meno, vek a pohlavie a výsledok vráťte ako objekt typu `Series`.

In [100]:
# solution
fltr = passengers['Age'] == passengers['Age'].min()
passengers.loc[ fltr, ['Name', 'Age'] ]

is_female = (passengers['Sex'] == 'female')
is_male = (passengers['Sex'] == 'male')

pd.concat([
    # youngest female
    passengers.loc[ is_female, ['Name', 'Age', 'Sex']] \
        .nsmallest(1, 'Age', keep='all') \
        .dropna()
,
    # youngest male
    passengers.loc[ is_male, ['Name', 'Age', 'Sex']] \
        .nsmallest(1, 'Age', keep='all') \
        .dropna()
])

,Name,Age,Sex
PassengerId,,,
1246,"Dean, Miss. Elizabeth Gladys Millvina""""",0.17,female
1093,"Danbom, Master. Gilbert Sigvard Emanuel",0.33,male


### 1.7 Aký bol priemerný vek pasažierov?

Už vieme, kto bol najmladší a kto bol najstarší, tak ešte zistime, aký bol priemerný vek pasažierov na lodi. Výsledok zaokrúhlite na 2 desatinné miesta.

In [111]:
# solution

passengers['Age'].mean().round(2)  # only pandas version >= 1.4
round(passengers['Age'].mean(), 2)

29.88

### 1.8 Zisk z predaja lístkov

Koľko peňazí tvoril zisk z plavby, ktorý sa vyzbieral z cestovného?

In [113]:
# solution
passengers['Fare'].sum()

43550.4869

### 1.9 Priemerná cena lístka

Rozličné triedy pasažierov zaplatili za lístok rozličnú sumu. Zistite, aká bola priemerná cena lístka pre každú triedu pasažierov.

In [116]:
passengers.groupby('Pclass')['Fare'].mean()

Pclass
1    87.508992
2    21.179196
3    13.302889
Name: Fare, dtype: float64

### 1.10 Jack a Rose

Keďže Cameroonovu verziu prvej a poslednej plavby Titanicu pozná zrejme každý, pozrime sa na to, ktorí cestujúci by mohli byť potenciálnymi predstaviteľmi jeho hrdinov. Takže zistite, koľko Jackov a koľko Rose bolo na palube Titanicu. Okrem ich mena vieme povedať, že:

* Jack bol muž
* Rose bola žena 
* žiadny z nich určite nemal viac ako 30 rokov
* Rose patrila do prvej triedy

In [142]:
# solution
is_jack = (passengers['Sex'] == 'male') \
    & (passengers['Age'] < 30) \
    & (passengers['Name'].str.contains('Jack'))

is_rose = (passengers['Sex'] == 'female') \
    & (passengers['Name'].str.contains('Rose')) \
    & (passengers['Age'] < 30) \
    & (passengers['Pclass'] == 1)

passengers.loc[is_rose | is_jack, ['Name', 'Age', 'Sex', 'Pclass']]

,Name,Age,Sex,Pclass
PassengerId,,,,


## 2. Nalodenie

### 2.1 Počet prístavov

Z koľkých prístavov nastupovali pasažieri? Vypíšte číslo zodpovedajúce počtu prístavov.

In [144]:
# solution
len(passengers.groupby('Embarked')['Embarked'])

3

### 2.2 Prehľad nalodených pasažierov

Vytvorte prehľad, z ktorého prístavu koľko ľudí nastúpilo na palubu. Výsledný objekt typu `Series` nech má ako index názvy prístavov, z ktorých sa pasažieri naloďovali.

In [159]:
# solution
passengers.groupby('Embarked')['Embarked'] \
    .count() \
    .rename({
        'S': 'Southampton',
        'Q': 'Queenstown',
        'C': 'Cherbourg'
    })

Embarked
Cherbourg      270
Queenstown     123
Southampton    914
Name: Embarked, dtype: int64

### 2.3 Najväčší prístav

A z ktorého prístavu nastúpilo najviac pasažierov? Vypíšte jeho názov a počet cestujúcich.

In [180]:
# solution
passengers.groupby('Embarked')['Embarked'] \
    .count() \
    .rename({
        'S': 'Southampton',
        'Q': 'Queenstown',
        'C': 'Cherbourg'
    }) \
    .sort_values() \
    .tail(1)


passengers.groupby('Embarked')['Embarked'] \
    .count() \
    .rename({
        'S': 'Southampton',
        'Q': 'Queenstown',
        'C': 'Cherbourg'
    }) \
    .nlargest(1)

Embarked
Southampton    914
Name: Embarked, dtype: int64

### 2.4 Najviac detí

Z ktorého prístavu nastúpilo najviac detí? Vypíšte jeho názov a počet detí.

In [186]:
# solution
is_kid = passengers['Age'] < 15

passengers \
    .loc[is_kid, :] \
    .groupby('Embarked')['Embarked'] \
    .count() \
    .rename({
        'S': 'Southampton',
        'Q': 'Queenstown',
        'C': 'Cherbourg'
    }) \
    .nlargest(1)

Embarked
Cherbourg      21
Queenstown      5
Southampton    83
Name: Embarked, dtype: int64

## 3. O preživších

O tom, kto prežil, máme tiež záznamy. Jednak nás budú zaujímať absolútne čísla, ale druhak nás bude zaujímať aj to, čo všetko mohlo mať vplyv na to, že dotyčný človek prežil. A či nemali vyššiu pravdepodobnosť prežitia tí, ktorí patrili do vyššej vrstvy alebo boli mladší.

In [211]:
# filters
survived = passengers['Survived'] == 1
is_male = passengers['Sex'] == 'male'
is_kid = passengers['Age'] < 15
is_rich = passengers['Pclass'] == 1

### 3.1 Koľko ľudí katastrofu prežilo?

Zistite celkový počet ľudí, ktorí prežili potopenie *Titanic*-u. Okrem celkového počtu uveďte tento počet aj percentuálne.

In [201]:
# solution
total_passengers = len(passengers['Survived'].dropna())
total_survived = len(passengers.loc[survived])

print(f'{total_survived}/{total_passengers} ({total_survived/total_passengers*100:.4}%)')

342/891 (38.38%)


### 3.2 Preživší a ich pohlavie

Malo na prežitie vplyv aj pohlavie? Koľko prežilo vo výsledku mužov a koľko žien? A koľko to bolo z celkového počtu mužov a žien?


In [208]:
# solution
total_males = len(passengers.loc[is_male])
total_females = len(passengers.loc[~is_male])

total_males, total_females, total_males+total_females

survived_males = len(passengers.loc[is_male & survived])
survived_females = len(passengers.loc[~is_male & survived])

print(f'males: {survived_males}/{total_males} ({survived_males/total_males*100:.4}%)')
print(f'females: {survived_females}/{total_females} ({survived_females/total_females*100:.4}%)')

males: 109/843 (12.93%)
females: 233/466 (50.0%)


### 3.3 Preživší a ich vek

Mal na prežitie vplyv aj vek pasažierov? Koľko prežilo vo výsledku detí a koľko dospelých? A koľko to bolo z celkového počtu detí a dospelých?

In [210]:
# solution
total_kids = len(passengers.loc[is_kid])
total_adults = len(passengers.loc[~is_kid])

survived_kids = len(passengers.loc[is_kid & survived])
survived_adults = len(passengers.loc[~is_kid & survived])

print(f'kids: {survived_kids}/{total_kids} ({survived_kids/total_kids*100:.4}%)')
print(f'adults: {survived_adults}/{total_adults} ({survived_adults/total_adults*100:.4}%)')

kids: 45/109 (41.28%)
adults: 297/1200 (24.75%)


### 3.4 Preživší a ich trieda

Mala na prežitie vplyv aj spoločenská trieda pasažierov? Koľko prežilo vo výsledku chudákov a koľko zámožných pasažierov? A koľko to bolo z celkového počtu cestujúcich?

In [216]:
# solution
total_rich = len(passengers.loc[is_rich])
total_poor = len(passengers.loc[~is_rich])

survived_rich = len(passengers.loc[is_rich & survived])
survived_poor = len(passengers.loc[~is_rich & survived])

print(f'{survived_rich}/{total_rich} ({survived_rich/total_rich*100:.4}%)')
print(f'{survived_poor}/{total_poor} ({survived_poor/total_poor*100:.4}%)')

136/323 (42.11%)
206/986 (20.89%)
